In [2]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
import warnings
warnings.filterwarnings('ignore')


In [3]:
USE_HUGGINGFACE = True

In [4]:
MODEL_PATH = "didulantha/sms-spam-detector"

In [5]:
try:
    classifier = pipeline("text-classification", model=MODEL_PATH)
    print("✓ Model loaded successfully!\n")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nMake sure:")
    print("1. Model is trained and saved in ./sms-spam-detector/")
    print("2. Or update MODEL_PATH with your HuggingFace model ID")
    exit(1)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


✓ Model loaded successfully!

